In [48]:
import numpy as np
import matplotlib.pyplot as plt
import alpha_recoil_sim as ar
import importlib, glob, os, shutil, subprocess, pickle

In [28]:
## first make a list of all the daughter isotopes we need
path = "/Users/dcmoore/Library/CloudStorage/GoogleDrive-david.c.moore@yale.edu/My Drive/yale/uspheres/alpha_recoils_Grimm/Sphere_Recoils_MC"

iso_list = ['Ac-225', 'Pb-212', 'At-211', 'Th-227', 'Ra-223']

iso_dict = {}

for iso in iso_list:
    ciso = iso[:2].lower()
    cA = iso[-3:]
    iso_dict[iso] = ar.parse_decay_chain(path + "/decay_data/" + ciso + "_" + cA + "_decay_chain.txt")

In [29]:
## go through the dictionary and make a list of all alpha decay daughters we need SRIM simulations for
daughter_list = []
for iso in iso_list:
    curr_dict = iso_dict[iso]
    
    curr_keys = curr_dict.keys()

    for k in curr_keys:
        if not '_daughters' in k: continue

        curr_daught = curr_dict[k]
        curr_alpha = curr_dict[k[:6] + "_decays"][:,1] > 0

        for j in range(len(curr_daught)):
            if( curr_alpha[j] and not curr_daught[j] in daughter_list):
                daughter_list.append(curr_daught[j]) 

print(daughter_list)


['Fr-221', 'At-217', 'Bi-213', 'Tl-209', 'Pb-209', 'Tl-208', 'Pb-208', 'Bi-207', 'Pb-207', 'Ra-223', 'Rn-219', 'Po-215', 'Pb-211', 'Tl-207']


In [30]:
## now generate the TRIM input files from the templates for the list of isotopes above
importlib.reload(ar)

material_list = ["SiO2", "Au", "Ag"]
recoil_energy = 200 ## keV
num_events = 10000 ## number of events for each isotope and material

fidx = 0
for mat in material_list:

    template_file = path + "/TRIM_input_files/TRIM.IN_%s.txt"%mat

    with open(template_file, 'r') as tf:
        template_lines = tf.readlines()

    for iso in daughter_list:

        output_file = path + "/TRIM_input_files/TRIM.IN_%d"%fidx

        with open(output_file, 'wt') as of:
            
            for lidx, l in enumerate(template_lines):

                if lidx == 0: ## write the first line as a header unchanged
                    of.write(template_lines[0])
                    continue

                ## update ion information
                if template_lines[lidx - 1].startswith("Ion:"):
                    Z, A = ar.get_Z_A_for_iso(iso)
                    ion_line = "    %d    %d    %d    0    %d    1    %d\n"%(Z, A, recoil_energy, num_events, num_events)
                    of.write(ion_line)
                elif template_lines[lidx - 1].startswith("Target material"):
                    new_start = "%s (%d) into "%(iso, recoil_energy)
                    newl = l[0] + new_start + l[14:]
                    of.write(newl)
                else:
                    of.write(l)

        fidx += 1


In [31]:
## code to run on windows computer to run TRIM jobs
input_files = glob.glob("C:/SRIM/Sphere_Recoils_MC/TRIM_input_files/*")

replace_files = False

destination_file = r"C:/SRIM/TRIM.IN"
trim_path = r"C:\SRIM"
output_dir = r"C:\SRIM\output_files_batch"
exyz_file = r"C:\SRIM\SRIM Outputs\EXYZ.txt"

for file in input_files:
    if file.endswith('.txt'): continue

    file_idx = file.split("_")[-1]

    if(os.path.isfile(output_dir + "\exyz_%s.txt"%file_idx) and not replace_files):
        print(output_dir + "\exyz_%s.txt"%file_idx + " already exists, skipping")
        continue

    shutil.copy(file, destination_file)
    print("Copied %s to %s"%(file, destination_file)) 

    print(trim_path)
    os.chdir(trim_path)
    subprocess.call("./TRIM.exe")

    shutil.copy(exyz_file, output_dir + "\exyz_%s.txt"%file_idx)


In [50]:
## parse exyz.txt files from TRIM and save in more efficient python data structure

data_path = '/Users/dcmoore/Library/CloudStorage/GoogleDrive-david.c.moore@yale.edu/My Drive/yale/uspheres/alpha_recoils_Grimm/SRIM_Data/'

data_dict = {}

A_to_nm = 0.1 #convert angstrom to nm

fidx = 0
for mat in material_list:
    for iso in daughter_list:

        with open(data_path + "exyz_%d.txt"%fidx) as df:
            lines = df.readlines()
        
        event_dict = {}
        curr_event = -1
        curr_traj_data = []
        for l in lines:
            if not l.startswith("0"): continue 

            curr_dat = l.strip().split()
            line_event = int(curr_dat[0])
            if(curr_event == -1):
                curr_event = line_event

            if( abs(line_event - curr_event) < 0.1):
                curr_traj_data.append([float(curr_dat[1]), float(curr_dat[2])*A_to_nm, float(curr_dat[3])*A_to_nm, float(curr_dat[4])*A_to_nm])
            else:
                if( line_event % 2000 == 0 ): 
                    print("Working on %s in %s, event %d"%(iso, mat, line_event))
                event_dict[curr_event] = np.array(curr_traj_data, dtype=float)
                curr_event = line_event
                curr_traj_data = [[float(curr_dat[1]), float(curr_dat[2])*A_to_nm, float(curr_dat[3])*A_to_nm, float(curr_dat[4])*A_to_nm],]

        data_dict[iso + "_" + mat] = event_dict


    fidx += 1

with open(data_path + 'SRIM_MC_events.pkl', 'wb') as f:
    pickle.dump(data_dict, f)
    

Working on Fr-221 in SiO2, event 2000
Working on Fr-221 in SiO2, event 4000
Working on Fr-221 in SiO2, event 6000
Working on Fr-221 in SiO2, event 8000
Working on Fr-221 in SiO2, event 10000
Working on At-217 in SiO2, event 2000
Working on At-217 in SiO2, event 4000
Working on At-217 in SiO2, event 6000
Working on At-217 in SiO2, event 8000
Working on At-217 in SiO2, event 10000
Working on Bi-213 in SiO2, event 2000
Working on Bi-213 in SiO2, event 4000
Working on Bi-213 in SiO2, event 6000
Working on Bi-213 in SiO2, event 8000
Working on Bi-213 in SiO2, event 10000
Working on Tl-209 in SiO2, event 2000
Working on Tl-209 in SiO2, event 4000
Working on Tl-209 in SiO2, event 6000
Working on Tl-209 in SiO2, event 8000
Working on Tl-209 in SiO2, event 10000
Working on Pb-209 in SiO2, event 2000
Working on Pb-209 in SiO2, event 4000
Working on Pb-209 in SiO2, event 6000
Working on Pb-209 in SiO2, event 8000
Working on Pb-209 in SiO2, event 10000
Working on Tl-208 in SiO2, event 2000
Working

In [47]:
data_dict['Fr-221_SiO2'][9901]

array([[ 2.0000e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00],
       [ 1.9878e+02,  9.5527e+00, -1.3213e-01,  3.0225e-02],
       [ 1.9793e+02,  2.9262e+01, -5.7465e-01,  1.4326e-01],
       [ 1.9691e+02,  3.9105e+01, -6.5288e-01,  1.7979e-01],
       [ 1.9590e+02,  4.8936e+01, -7.4508e-01,  1.8897e-01],
       [ 1.9479e+02,  7.8384e+01, -1.0913e+00,  4.1212e-01],
       [ 1.9370e+02,  9.3085e+01, -1.3168e+00,  6.2625e-01],
       [ 1.9284e+02,  1.0287e+02, -1.4047e+00,  7.8703e-01],
       [ 1.9135e+02,  1.1754e+02, -1.4700e+00,  1.2332e+00],
       [ 1.8841e+02,  1.3217e+02, -1.3598e+00,  1.7102e+00],
       [ 1.8792e+02,  1.3703e+02, -1.4070e+00,  1.7778e+00],
       [ 1.8675e+02,  1.5646e+02, -1.6261e+00,  2.0600e+00],
       [ 1.8468e+02,  1.6616e+02, -1.5865e+00,  2.3671e+00],
       [ 1.8367e+02,  1.8065e+02, -1.5995e+00,  3.1008e+00],
       [ 1.8253e+02,  1.9030e+02, -1.6362e+00,  3.6616e+00],
       [ 1.5682e+02,  1.9511e+02, -1.6786e+00,  3.9673e+00],
       [ 1.5593e+02,  2.